In [3]:
import torchvision.models as models

# Load the pre-trained SqueezeNet model
model = models.squeezenet1_1(pretrained=True)

ModuleNotFoundError: No module named 'torchvision'

In [2]:
print(model)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [3]:
import torch
from torchvision import transforms
from torchvision.models import squeezenet1_1
from PIL import Image
import time

# Load the pre-trained SqueezeNet model
model = squeezenet1_1(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Load and preprocess an example image
image_path = '1.jpg'
input_image = Image.open(image_path).convert('RGB')

# Define the image transformation pipeline
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply transformations to the input image
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

# Make the prediction
with torch.no_grad():
    start_time = time.time()
    output = model(input_batch)
    end_time = time.time()

# Get the predicted class index
_, predicted_idx = torch.max(output, 1)
predicted_class = "Cats" if predicted_idx.item() == 0 else "Dogs"

# Print the predicted class label and inference time
inference_time = end_time - start_time
print(f'Predicted class: {predicted_class}')
print(f'Inference time: {inference_time:.4f} seconds')

Predicted class: Dogs
Inference time: 0.1488 seconds


In [4]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total trainable parameters: {total_params}')

Total trainable parameters: 1235496


In [5]:
import torch
import torchvision.models as models

# Load the pre-trained SqueezeNet model
model = models.squeezenet1_1(pretrained=True)

# Define input size (224x224x3) for SqueezeNet
input_size = (224, 224, 3)

# Function to calculate FLOPs for the model
def count_flops(model, input_size):
    # Define a tensor of appropriate size
    input_tensor = torch.randn(1, *input_size)
    
    # Switch to evaluation mode
    model.eval()
    
    # Move model to appropriate device
    device = next(model.parameters()).device
    input_tensor = input_tensor.to(device)
    model.to(device)
    
    # Iterate through model's layers
    flops = 0
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            # For convolutional layers
            output_size = (input_size[0] - module.kernel_size[0] + 2 * module.padding[0]) // module.stride[0] + 1
            output_size = (output_size - module.kernel_size[1] + 2 * module.padding[1]) // module.stride[1] + 1
            flops += module.in_channels * module.out_channels * module.kernel_size[0] * module.kernel_size[1] * output_size * output_size
            input_size = (output_size, output_size, module.out_channels)
        elif isinstance(module, torch.nn.MaxPool2d):
            # For max pooling layers
            output_size = (input_size[0] - module.kernel_size) // module.stride + 1
            flops += input_size[2] * output_size * output_size
            input_size = (output_size, output_size, input_size[2])
        elif isinstance(module, torch.nn.Linear):
            # For fully connected layers
            flops += module.in_features * module.out_features
            input_size = (module.out_features,)
    
    return flops

# Calculate FLOPs
total_flops = count_flops(model, input_size)
print("Total FLOPs:", total_flops)

Total FLOPs: 78414144
